In [97]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np

# Excel data as pandas dataframe
#filepath = r'C:\Users\Agni\Documents\MIT 2017-18\6.884\neuro-soc\Data\neuro_dataset.xlsx'
filepath = r'C:\Users\Agni\Documents\MIT 2017-18\6.884\neuro-soc\Data\neuro_compiled.xlsx'
df_original = pd.read_excel(filepath, sheetname=0)
#print(df_original)
#print(df_original['Hospital Type (DH)'])

In [98]:
# Categorical to numerical dataframe (contains only the originally categorical columns)
categorical_cols_df = df_original[['Time Zone', 'State', 'SEX', 'Service Line', 'Reason for Consult', 'Provider Diagnosis', 'Hospital Type (DH)', 'Stroke Center',  'Advanced Comprehensive Stroke Center']].copy()
#print(categorical_cols_df)

categorical_numerical_df = categorical_cols_df.apply(lambda x: pd.factorize(x)[0])
#print(categorical_numerical_df)

In [99]:
# COMPLETE INPUT DATAFRAME
# Input features (numerical cols from originally categorical ones, also originally numerical ones)
# ['Visit Initiated', 'Age', 'Bed Count', 'Total ER Visits (DH)']

all_inputs_df = categorical_numerical_df.copy()
all_inputs_df['Visit Initiated'] = df_original['Visit Initiated']
all_inputs_df['Age'] = df_original['Age']
all_inputs_df['Bed Count'] = df_original['Bed Count']
all_inputs_df['Total ER Visits (DH)'] = df_original['Total ER Visits (DH)']

all_inputs_df['Visit Initiated (Float)'] = df_original['Visit Initiated (Float)']

print(all_inputs_df)

       Time Zone  State  SEX  Service Line  Reason for Consult  \
0              0      0    0             0                  -1   
1              0      1    0             0                  -1   
2              1      2    0             0                  -1   
3              2      3    1             0                  -1   
4              2      3    0             0                  -1   
5              0      0    1             1                  -1   
6              1      4    1             1                  -1   
7              2      3    1             0                  -1   
8              2      3    1             0                  -1   
9              0      5    0             1                  -1   
10             2      3    1             0                  -1   
11             1      6    1             1                  -1   
12             2      3    0             1                  -1   
13             2      3    0             0                  -1   
14        

In [100]:
# GENERATING y COLUMN (with 136 entries for 136 weeks)
# New dataframe with demand quantity (by week) column

df = df_original.copy() # for weekly examination
df['Name'] = 'Patient'
df['Quantity'] = 1
df['Visit Initiated'] = pd.to_datetime(df['Visit Initiated']) - pd.to_timedelta(7, unit='d')
df = df.groupby(['Name', pd.Grouper(key='Visit Initiated', freq='W-MON')])['Quantity'].sum().reset_index().sort_values('Visit Initiated')
df['date_delta'] = (df['Visit Initiated'] - df['Visit Initiated'].min())  / np.timedelta64(1,'D')

#quantity_df = df['date_delta']
quantity_df = df.filter(['Quantity'], axis=1)
#print(quantity_df)

In [101]:
# GENERATING complete y COLUMN with script 

def populate(df_input, col):
    """ 
    df_input: input dataframe
    col: string name of the column aggregating
    """
    df = df_input.copy() # don't modify original
    a = df[col].values.T.tolist()
    b = []
    for item in a:
        item = int(item)
        for i in range(item):
            b.append(item)
    # convert b back into a DataFrame
    df_output = pd.DataFrame(b, columns = list(df.columns.values))
    return df_output

quantity_df_extended = populate(quantity_df, 'Quantity')
#print(quantity_df_extended)

In [114]:
#y = df['Quantity'].copy()
#y = df_original
#print(y.shape)

#print(all_inputs_df) #95256 rows x 14 columns
#print(quantity_df_extended) #95256 rows x 1 columns

combined_df = pd.concat([all_inputs_df, quantity_df_extended], axis=1)
#print(combined_df) #95256 rows x 15 columns

new_all_inputs = (combined_df.dropna(axis=0, how='any')).drop(['Visit Initiated'], axis=1)
#print(new_all_inputs)

y = new_all_inputs['Quantity']
x_inputs = new_all_inputs.drop(['Quantity'], axis=1)

# Create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(x_inputs, y, test_size=0.2)
#print(X_train.shape, y_train.shape)
#print(X_test.shape, y_test.shape)

print("---------------------------")
print("X_train:", X_train)
#print("y_train:", y_train)
#print("X_test:", X_test)
#print("y_test:", y_test)

---------------------------
X_train:        Time Zone  State  SEX  Service Line  Reason for Consult  \
60243          1      4    0             1                  74   
41756          1      8    1             1                  81   
63110          0     15    1             1                  78   
79451          1      2    0             1                  74   
67042          0     21    1             1                   5   
36538          1      5    0             1                  74   
45041          4     12    1             1                  74   
45173          3      0    0             1                  74   
44464          1      7    1             0                   4   
43346          0      0    1             0                   5   
95106          1      8    1             1                   7   
84822          0      0    1             1                  75   
89254          1     16    1             1                  74   
85776          1      8    0           

In [115]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [116]:
# fit a model
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

print(predictions[0:5])

print("Score:", model.score(X_test, y_test))

## The line / model
plt.scatter(y_test, predictions)
plt.xlabel("True Values")
plt.ylabel("Predictions")

# Cross-validation (to be entered below)

[ 872.71538673  737.225061    951.89534076  908.86582759  901.94603817]
Score: 0.782112663466


In [ ]:

#print(all_inputs_df.drop(['Visit Initiated']))